In [27]:
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
from dash.dependencies import Input, Output 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
psw='navidadp3'

1. Grafica de barras

In [39]:

def categ_año():
    # Conectarse a la base de datos usando SQLAlchemy
    engine = psycopg2.connect(
        dbname="world",
        user="postgres",
        password=psw,  # Asegúrate de definir la variable 'psw' antes de usarla
        host="p3.curxufagptbe.us-east-1.rds.amazonaws.com",
        port='5432'
    )
    cursor = engine.cursor()

    # Consulta SQL para contar la proporción de puntajes por año
    query = """
    SELECT periodo, 
       punt_global AS "Cuartil",  -- Cambiar el nombre de la columna
       COUNT(*) AS cantidad
       FROM bd_q
       WHERE periodo IN (2019, 2022)
       GROUP BY periodo, punt_global;
    """
    
    # Ejecutar la consulta y obtener el resultado como un DataFrame
    df = pd.read_sql_query(query, engine)

    # Mapear los valores de punt_global según la asignación dada
    mapping = {
        1: 'Cuartil 1',
        2: 'Cuartil 2',
        3: 'Cuartil 3',
        4: 'Cuartil 4'
    }
    
    # Aplicar el reemplazo
    df['Cuartil'] = df['Cuartil'].replace(mapping)

    # Filtrar solo los años que te interesan
    df = df[df['periodo'].isin([2019, 2022])]
    
    # Calcular la proporción de cada categoría en relación con la cantidad total por año
    df['proporcion'] = df.groupby('periodo')['cantidad'].transform(lambda x: x / x.sum())

    # Crear la figura de la gráfica de barras
    fig = px.bar(
        df, 
        x='periodo', 
        y='proporcion',
        color='Cuartil',
        labels={'proporcion': 'Porcentaje', 'periodo': 'Año'},
        title='Porcentaje de notas ICFES por cuartiles antes y después de pandemia en Magdalena',
        category_orders={'Cuartil': ['Cuartil 1', 'Cuartil 2', 'Cuartil 3', 'Cuartil 4']},
        color_discrete_sequence=px.colors.sequential.Greens_r,
        text=df['proporcion']
    )  
    
    # Personalizar el diseño de la gráfica
    fig.update_xaxes(title_text='', showticklabels=True, categoryorder='array', categoryarray=[2019,2022])  # Ocultar etiquetas y valores del eje x
    fig.update_yaxes(showgrid=False, showline=False, zeroline=False, title_text='', showticklabels=False)  # Ocultar etiquetas y valores del eje y

    # Personalizar el formato del texto y centrarlo
    fig.update_traces(texttemplate='%{text:.2%}', 
                      textfont=dict(color='white', size=18, family='Arial, sans-serif'), 
                      insidetextanchor='start',
                      textposition='inside')

    # Crear la aplicación Dash
    app = dash.Dash(__name__)

    app.layout = html.Div([
        dcc.Graph(figure=fig)
    ])

    # Ejecutar la aplicación Dash
    if __name__ == '__main__':
        app.run_server(debug=True)

# Asegúrate de tener definida la variable 'psw' antes de llamar a la función
categ_año()



C:\Users\Sebastian\AppData\Local\Temp\ipykernel_496\4207886967.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

